<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width="35%" align="right">


# yoctoGPT — Finance Text Corpus (Token Mode)

This notebook trains `yoctoGPT` on a **finance text corpus** from the `finance/` folder.
It currently includes `financial_theory.txt`, and can scale to multiple finance `.txt` files later.
Target setup: **Google Colab + L4 GPU**.

## How to Use This Notebook

1. Run setup and ensure at least one `.txt` exists in `finance/`.
2. Build tokenizer + dataset from all `finance/*.txt` files.
3. Train with L4-friendly defaults.
4. Sample and score text quality with readability metrics.
5. Optionally resume from `best.pt` with a lower LR.

### Roadmap
- Setup and environment checks
- Multi-file finance tokenization (`finance/*.txt`)
- L4 training run
- Sampling + readability scoring
- Resume from best checkpoint

In [ ]:
#@title Mount Google Drive for checkpoint storage (repo stays local)
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

CKPT_DIR = Path('/content/drive/MyDrive/yocto/checkpoints/financial_theory_token')
CKPT_DIR.mkdir(parents=True, exist_ok=True)
print('Checkpoints dir:', CKPT_DIR)

In [ ]:
#@title Setup: Install Dependencies and Clone Repository
!nvidia-smi || true
!pip -q install tokenizers tqdm textstat

import os
import pathlib
import subprocess

repo_root = pathlib.Path('/content/yoctoGPT')
if repo_root.exists():
    print('Repo exists, pulling latest...')
    subprocess.run(['git', 'pull'], cwd=repo_root, check=False)
else:
    subprocess.run(['git', 'clone', 'https://github.com/yhilpisch/yoctoGPT.git', str(repo_root)], check=False)
os.chdir(repo_root)

if os.path.exists('requirements.txt'):
    !pip -q install -r requirements.txt || true

finance_dir = pathlib.Path('finance')
finance_dir.mkdir(exist_ok=True)
txts = sorted(finance_dir.glob('*.txt'))
if not txts:
    raise FileNotFoundError('No .txt files found in finance/. Add at least one corpus file.')

print(f'Found {len(txts)} finance text file(s):')
for fp in txts[:10]:
    print(' -', fp.name)
if len(txts) > 10:
    print(' - ...')

### Tokenization

For technical finance text (formulas, symbols, and code-like fragments), token-level modeling is typically more effective than pure character-level modeling.

We build one tokenizer from **all files in `finance/`**.

In [ ]:
#@title Prepare Token-level Dataset from finance/*.txt
!python -m scripts.prepare_tokenizer \
--all_txt_in_dir \
--text_dir finance \
--out_dir data/token_finance \
--vocab_size 12000 \
--backend bpe \
--random_split \
--split_seed 1337 \
--add_bos_eos

In [ ]:
#@title Pick L4-friendly block_size/batch_size
import numpy as np
from pathlib import Path

train_path = Path('data/token_finance/train.bin')
val_path = Path('data/token_finance/val.bin')
train_tokens = int(np.fromfile(train_path, dtype=np.int32).shape[0])
val_tokens = int(np.fromfile(val_path, dtype=np.int32).shape[0])
min_tokens = min(train_tokens, val_tokens)

block_candidates = [512, 384, 256, 192, 128, 96, 64, 48, 32, 24, 16]
block_size = next((b for b in block_candidates if min_tokens > b + 2), max(8, min_tokens - 2))

target_tokens = min(32768, max(2048, min_tokens))
batch_size = max(1, min(128, target_tokens // block_size))

print(f'Using block_size={block_size}, batch_size={batch_size}')

### Training

L4-oriented baseline: `gpt_fast` + bf16 + cosine schedule.

This should provide a good balance of speed and quality for one technical book corpus.

In [ ]:
#@title Train (gpt_fast) on Colab L4
from pathlib import Path

CKPT_DIR = Path('/content/drive/MyDrive/yocto/checkpoints/finance_token')
CKPT_DIR.mkdir(parents=True, exist_ok=True)

!python -m yoctoGPT.train \
--mode token \
--data_dir data/token_finance \
--tokenizer_path data/token_finance/tokenizer.json \
--ckpt_dir {CKPT_DIR} \
--model_type gpt_fast \
--device cuda \
--n_layer 6 --n_head 6 --n_embd 384 \
--block_size {block_size} --batch_size {batch_size} \
--dropout 0.12 --weight_decay 0.08 \
--tie_weights --label_smoothing 0.05 \
--amp --amp_dtype bf16 \
--auto_microbatch \
--eval_interval 250 --eval_iters 30 \
--cosine_lr --warmup_iters 200 \
--min_lr 1e-5 --lr 1.5e-4 \
--max_iters 3000 \
--ema --ema_decay 0.999

### Sampling and Resuming

Sample from `best.pt` and score readability. Then optionally resume from `best.pt` with a lower LR.

In [ ]:
#@title Sample Continuation
output_token = !python -m yoctoGPT.sampler \
--mode token \
--ckpt {CKPT_DIR}/best.pt \
--tokenizer_path data/token_finance/tokenizer.json \
--prompt "in financial theory, the principle of no arbitrage implies" \
--max_new_tokens 150 \
--temperature 0.9 --top_k 30 --top_p 0.9

generated_text_token = '\n'.join(output_token)
print(generated_text_token)

### Readability Assessment

Use the same readability scoring as other notebooks for direct comparison of output quality.

In [ ]:
#@title Analyze Generated Text Readability
from textstat import textstat

def readability_scores(text: str) -> dict:
    return {
        'Flesch Reading Ease': textstat.flesch_reading_ease(text),
        'Flesch-Kincaid Grade': textstat.flesch_kincaid_grade(text),
        'Dale-Chall Score': textstat.dale_chall_readability_score(text),
        'Text Standard': textstat.text_standard(text, float_output=False),
    }

scores = readability_scores(generated_text_token)
for metric, value in scores.items():
    print(f'{metric:20}: {value}')

In [ ]:
#@title Resume training from best.pt (lower LR + early stopping)
best = CKPT_DIR / 'best.pt'
if not best.exists():
    print('No best.pt found; skipping resume cell.')
else:
    !python -m yoctoGPT.train \
      --mode token \
      --data_dir data/token_finance \
      --tokenizer_path data/token_finance/tokenizer.json \
      --ckpt_dir {CKPT_DIR} \
      --resume {best} \
      --model_type gpt_fast \
      --device cuda \
      --n_layer 6 --n_head 6 --n_embd 384 \
      --block_size {block_size} --batch_size {batch_size} \
      --dropout 0.12 \
      --tie_weights \
      --amp --amp_dtype bf16 \
      --auto_microbatch \
      --cosine_lr --warmup_iters 80 \
      --lr 8e-5 --max_iters 1200 \
      --eval_interval 250 --eval_iters 80 \
      --early_stopping_patience 3 \
      --early_stopping_min_delta 0.01

### Exercises

1. **Code-Like Behavior**: Try prompts that start with Python syntax (e.g., `def black_scholes(`) and compare outputs across temperatures.
2. **Context Length**: Compare `block_size=384` vs `512` for coherence around formulas and multi-line explanations.
3. **Architecture Comparison**: Train a shorter run with `--model_type gpt_plus` and compare validation loss and sample quality.

<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width="35%" align="right">
